In [1]:
%run QM9GNN2_Multitask.ipynb

ModuleNotFoundError: No module named 'keras'

In [12]:
import itertools

In [13]:
optimizer = Adam(lr=learning_rate)
loss = 'mse'

In [14]:
def generate_single_task_filename(task):
    return path.join('single_task_trained_models', task + '.h5')

In [15]:
# for i in range(-2, 0):
#     print('learning', tasks[i])
#     model = create_single_task_model(X_in, A_in, E_in)
#     model.compile(optimizer=optimizer, loss=loss)
#     es_callback = EarlyStopping(monitor='val_loss', patience=es_patience)
#     model.fit([X_train, A_train, E_train],
#              y_train_list[i],
#              batch_size=batch_size,
#              validation_split=0.1,
#              epochs=epochs,
#              callbacks=[es_callback])
#     model.save_weights(generate_single_task_filename(tasks[i]))

In [16]:
def load_and_compile_model(task):
    model = create_single_task_model()
    model.load_weights(generate_single_task_filename(task))
    model.compile(optimizer=optimizer, loss=loss)

In [17]:
def calculate_transfer_coefficient_from_file(task_learned, task_transferred, y_transferred_test):
    model_learned = create_single_task_model()
    model_transferred = create_single_task_model()
    model_learned.load_weights(generate_single_task_filename(task_learned))
    model_transferred.load_weights(generate_single_task_filename(task_transferred))
    
    layers_learned = model_learned.get_weights()
    layers_transferred = model_transferred.get_weights()    
    
    # 10 is a hard-coded value dependent on the architecture
    layers_transferred = layers_learned[:10] + layers_transferred[10:]
    model_transferred.set_weights(layers_transferred)
    model_transferred.compile(optimizer=optimizer, loss=loss)
    
    eval_results = model_transferred.evaluate([X_test, A_test, E_test],
                                              y_transferred_test,
                                              batch_size=batch_size)
    return eval_results

In [18]:
def load_and_test_single_task_model(task_learned, y_test):
    model = create_single_task_model()
    model.load_weights(path.join('single_task_trained_models', task_learned + '.h5'))
    model.compile(optimizer=optimizer, loss=loss)
    eval_results = model.evaluate([X_test, A_test, E_test], y_test, batch_size=batch_size)
    return eval_results

In [19]:
print(load_and_test_single_task_model('A', y_test_list[0]))
print(calculate_transfer_coefficient_from_file('A', 'B', y_test_list[1]))

1000/1000 [==============================] - 4s 4ms/step
2.84445166182518
1000/1000 [==============================] - 5s 5ms/step
0.1826805809736252


In [23]:
for task, test_data in zip(tasks, y_test_list):
    print(task, load_and_test_single_task_model(task, test_data))

1000/1000 [==============================] - 5s 5ms/step
A 2.84445166182518
1000/1000 [==============================] - 5s 5ms/step
B 0.21984173798561096
1000/1000 [==============================] - 5s 5ms/step
C 0.0553673215508461
1000/1000 [==============================] - 5s 5ms/step
mu 0.8005966105461121
1000/1000 [==============================] - 5s 5ms/step
alpha 3.969019329071045
1000/1000 [==============================] - 5s 5ms/step
homo 0.5102023949623108
1000/1000 [==============================] - 5s 5ms/step
lumo 0.3654808156490326
1000/1000 [==============================] - 5s 5ms/step
gap 0.6127849950790405
1000/1000 [==============================] - 5s 5ms/step
r2 2.7605866527557374
1000/1000 [==============================] - 5s 5ms/step
zpve 1.3736374816894532
1000/1000 [==============================] - 5s 5ms/step
u0 3.513526659011841
1000/1000 [==============================] - 5s 5ms/step
u298 3.6118437042236327
1000/1000 [==============================] - 5

In [24]:
for cluster in clusters:
    losses = load_and_evaluate_multitask_model(cluster, y_test_list)
    for task, loss in zip(cluster, losses):
        print(task, loss)

1000/1000 [==============================] - 8s 8ms/step
A 2.458219604969025
B 2.0499112951755523
alpha 0.2887927241325378
1000/1000 [==============================] - 8s 8ms/step
C 0.8096435604095459
r2 0.53923184800148
u0 0.26782286739349365
1000/1000 [==============================] - 8s 8ms/step
zpve 0.11785562551021576
g298 0.049128104984760286
cv 0.004560508601367474
1000/1000 [==============================] - 8s 8ms/step
lumo 0.13410502499341964
u298 0.13013995361328126
h298 0.0019254343137145042
1000/1000 [==============================] - 8s 8ms/step
mu 0.9110898313522339
homo 0.6530598673820496


In [24]:
print(tasks)
transfer_coefficient_dict = dict()
for i, j in itertools.permutations(range(len(tasks)), 2):
    task_learned = tasks[i]
    task_transferred = tasks[j]
    print(task_learned, task_transferred)
    transfer_coefficient = calculate_transfer_coefficient_from_file(task_learned, task_transferred, y_test_list[j])
    print(transfer_coefficient)
    
    if task_learned not in transfer_coefficient_dict.keys():
        transfer_coefficient_dict[task_learned] = {task_transferred: transfer_coefficient}
    else:
        transfer_coefficient_dict[task_learned][task_transferred] = transfer_coefficient

['A', 'B', 'C', 'mu', 'alpha', 'homo', 'lumo', 'gap', 'r2', 'zpve', 'u0', 'u298', 'h298', 'g298', 'cv', 'u0_atom', 'u298_atom', 'h298_atom', 'g298_atom']
A B
1000/1000 [==============================] - 2s 2ms/step
0.9776526494026184
A C
1000/1000 [==============================] - 2s 2ms/step
0.9291825671195983
A mu


KeyboardInterrupt: 

In [25]:
file = open('cross_task_transfer_coefficients.txt', 'w')
for key1, value in transfer_coefficient_dict.items():
    for key2, coef in value.items():
        print(key1, key2, coef)
        print(key1, key2, coef, file=file)

A B 0.9776526494026184
A C 0.9291825671195983


In [ ]:
# The rest of this file is comprised of mostly useless analysis code.

In [26]:
def analyze_transfer_coefficients():
    with open('cross_task_transfer_coefficients.txt', 'r') as file:
        lines = file.readlines()
        transfer_coefficient_dict = dict()
        for line in lines:
            task_learned, task_transferred, transfer_coefficient = line.strip().split()
            if task_learned not in transfer_coefficient_dict.keys():
                transfer_coefficient_dict[task_learned] = {task_transferred: float(transfer_coefficient)}
            else:
                transfer_coefficient_dict[task_learned][task_transferred] = float(transfer_coefficient)
        return transfer_coefficient_dict

In [23]:
transfer_coefficient = analyze_transfer_coefficients()
coef_list = list()
# relevant_tasks = ['alpha', 'homo', 'lumo', 'gap']
relevant_tasks = ['mu', 'homo']
for task_set in itertools.combinations(relevant_tasks, 3):
    coef_sum = 0
    for i, j in itertools.combinations(task_set, 2):
        coef_sum += transfer_coefficient[i][j]**2 + transfer_coefficient[j][i]**2
    coef_list.append((coef_sum, task_set))
# coef_list = np.asarray(coef_list)
# coef_list = np.sort(coef_list)
coef_list.sort()
for entry in coef_list:
    print(entry)

In [7]:
for i, j in itertools.combinations(transfer_coefficient.keys(), 2):
    if transfer_coefficient[i][j] < 0.9 and transfer_coefficient[j][i] < 0.9:
        print(i, 
              j, 
              round(transfer_coefficient[i][j], 3), 
              round(transfer_coefficient[j][i], 3))

A B 0.131 0.029
A C 0.041 0.09
B alpha 0.892 0.032
zpve u298_atom 0.736 0.864
u298 g298_atom 0.894 0.897
